sheet flow:

- ~~define track of interest based on sheet_gid dict~~
- ~~read in weekend dataframe~~
- ~~run function drop_empties~~
- ~~run function check_df~~
- ~~produce these lists:~~
    - ~~predicted_qualifying_order, actual_qualifying_order, predicted_race_order, actual_race_order~~
- ~~produce these dfs:~~
    - ~~constructor_pricing, driver_pricing~~
- ~~produce these lists:~~
    - ~~drivers, constructors~~
- ~~set up constants for tracking weekend scores~~
    - ~~constructor_scores, driver_scores~~
- ~~run function score_qualification_order~~
- run function score_race_order
- run function score_fastest_pitstop
- run function score_fastest_lap
    
still to-do:
- ~~changing the overtake bonus/gain/loss points from estimations to direct reports in fantasy~~
- ~~produce df of driver and constructor pricing for making team affordability calculations each week, later when i get to the full program part of it~~
- ~~incorporate the team recommendation function that the post i found used~~
- ~~write function scoring the predictions only~~
    - ~~assume difference between predicted qualifying position and predicted race position are direct gain/loss~~
- ~~incorporate the pieces into scoring all of the race into one function~~
    - ~~qualifying, race, superlatives, overtakes+gain/loss~~
- make google sheet of teams and track team composition through each weekend
    - read this in, figure out if i can define a range of cells so i can do all three teams on the same google sheet or if i'm going to have to do a separate sheet for each
- ~~add scoring for my team to use for the team recommendations~~
- ~~add ability to handle DNF, DNQ, and OUT (sainz, stroll, zhou at saudi arabia)~~

In [2]:
from weekend_functions import *

### Define Track and Read In Weekend Data

In [3]:
track_name = 'japan'

In [4]:
# read in from google sheets the sheet for this weekend's events
weekend_df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid={sheet_gid[track_name]}')

# read in from local sometimes needed when offline, make sure to download the file from sheets first...
# weekend_df = pd.read_csv(f'f1_main - {track_name}.csv')

weekend_df = drop_empties(weekend_df)
check_df(weekend_df)

weekend_df

,Team,Driver,predicted_qualifying_japan,predicted_race_japan
0,Red Bull Racing-RBPT,Max Verstappen,1,1
1,Ferrari,Charles Leclerc,2,3
2,Ferrari,Carlos Sainz Jr.,3,4
3,Red Bull Racing-RBPT,Sergio Perez,4,2
4,McLaren-Mercedes,Lando Norris,5,5
5,McLaren-Mercedes,Oscar Piastri,6,6
6,Mercedes,Lewis Hamilton,7,7
7,Mercedes,George Russell,8,8
8,Aston Martin Aramco-Mercedes,Fernando Alonso,9,9
9,Aston Martin Aramco-Mercedes,Lance Stroll,10,10


In [5]:
weekend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Team                        20 non-null     object
 1   Driver                      20 non-null     object
 2   predicted_qualifying_japan  20 non-null     int64 
 3   predicted_race_japan        20 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 768.0+ bytes


### Define Predicted Qualifying, Actual Qualifying, Predicted Race, Actual Race Orders, Driver Gain/Loss

this was used when writing functions to score the actual race. its not needed for what this sheet has become, picking a team for the next race. i'll move it to another sheet eventually when if/when i need it still. or i'll delete it.

In [5]:
# # qualifying orders
# predicted_qualifying_order = weekend_df[['Driver', f'predicted_qualifying_{track_name}']].sort_values(by=[f'predicted_qualifying_{track_name}'])['Driver'].tolist()
# actual_qualifying_order = weekend_df[['Driver', f'actual_qualifying_{track_name}']].sort_values(by=[f'actual_qualifying_{track_name}'])['Driver'].tolist()

# # race orders
# predicted_race_order = weekend_df[['Driver', f'predicted_race_{track_name}']].sort_values(by=[f'predicted_race_{track_name}'])['Driver'].tolist()
# if f'actual_race_{track_name}' in weekend_df.columns:
#     actual_race_order = weekend_df[['Driver', f'actual_race_{track_name}']].sort_values(by=[f'actual_race_{track_name}'])['Driver'].tolist()

# # driver_gain_loss
# driver_gain_loss_overtake = {}
# for driver in weekend_df.Driver:
#     driver_gain_loss_overtake[driver] = {}
#     driver_gain_loss_overtake[driver]['gain_loss'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'positions_gained_{track_name}']].values[0][0]
#     driver_gain_loss_overtake[driver]['overtake'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'overtake_bonus_{track_name}']].values[0][0]

### Driver and Constructor Prices

In [25]:
# pricing for drivers and constructors, updated each week with the latest values from the Formula 1 Fantasy Game
driver_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=920234107').dropna(axis=1, how='all')
constructor_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=1972297135').dropna(axis=1, how='all')

In [28]:
current_driver_values = {x[0]: x[1] for x in driver_pricing[['Driver', track_name]].values}
current_constructor_values = {x[0]: x[1] for x in constructor_pricing[['Constructor', track_name]].values}

### Look at Predictions for the Weekend

In [13]:
drivers, constructors, driver_scores, constructor_scores, driver_score_summary, constructor_score_summary = score_race_qualifying_predicted(weekend_df, track_name)

In [14]:
driver_scores

{'Max Verstappen': 45,
 'Charles Leclerc': 23,
 'Carlos Sainz Jr.': 19,
 'Sergio Perez': 29,
 'Lando Norris': 16,
 'Oscar Piastri': 13,
 'Lewis Hamilton': 10,
 'George Russell': 7,
 'Fernando Alonso': 4,
 'Lance Stroll': 2,
 'Yuki Tsunoda': 0,
 'Nico Hulkenberg': -1,
 'Kevin Magnussen': -1,
 'Daniel Ricciardo': 4,
 'Alexander Albon': 0,
 'Valtteri Bottas': 0,
 'Pierre Gasly': 0,
 'Logan Sargeant': 0,
 'Guanyu Zhou': 0,
 'Esteban Ocon': 0}

In [15]:
constructor_scores

{'Red Bull Racing-RBPT': 74,
 'Ferrari': 52,
 'McLaren-Mercedes': 39,
 'Mercedes': 27,
 'Aston Martin Aramco-Mercedes': 16,
 'VisaCashApp RB': 7,
 'Haas-Ferrari': 1,
 'Williams-Mercedes': 1,
 'Stake F1 Team Kick Sauber': -1,
 'Alpine-Renault': -1}

# Evaulate Next Race

In [30]:
# F-winning

my_constructors = ['Ferrari', 'Red Bull Racing-RBPT']
my_drivers = ['Daniel Ricciardo', 'Sergio Perez', 'Pierre Gasly', 'Yuki Tsunoda', 'Nico Hulkenberg']
remaining_cost_cap = 0.6

In [33]:
# my team predicted points and cost spent

team_score = sum(map(lambda x: driver_scores[x], my_drivers)) + max(map(lambda x: driver_scores[x], my_drivers)) + sum(map(lambda x: constructor_scores[x], my_constructors))
team_value = sum(map(lambda x: current_driver_values[x], my_drivers)) + sum(map(lambda x: current_constructor_values[x], my_constructors))

print(f'Team score with {my_drivers} and {my_constructors}: {team_score}')
print(f'Team value as it is currently constructed: {team_value}')

Team score with ['Daniel Ricciardo', 'Sergio Perez', 'Pierre Gasly', 'Yuki Tsunoda', 'Nico Hulkenberg'] and ['Ferrari', 'Red Bull Racing-RBPT']: 187
Team value as it is currently constructed: 103.4


In [11]:
top_teams = main(my_drivers, my_constructors, weekend_df, track_name, driver_pricing, constructor_pricing, remaining_cost_cap)

=== Predicted Driver Scores for Japan ===
{'Max Verstappen': 45, 'Charles Leclerc': 23, 'Carlos Sainz Jr.': 19, 'Sergio Perez': 29, 'Lando Norris': 16, 'Oscar Piastri': 13, 'Lewis Hamilton': 10, 'George Russell': 7, 'Fernando Alonso': 4, 'Lance Stroll': 2, 'Yuki Tsunoda': 0, 'Nico Hulkenberg': -1, 'Kevin Magnussen': -1, 'Daniel Ricciardo': 4, 'Alexander Albon': 0, 'Valtteri Bottas': 0, 'Pierre Gasly': 0, 'Logan Sargeant': 0, 'Guanyu Zhou': 0, 'Esteban Ocon': 0}

=== Current Team ===
Constructors: ['Ferrari', 'VisaCashApp RB']
Drivers: ['Daniel Ricciardo', 'Max Verstappen', 'Pierre Gasly', 'Oscar Piastri', 'Nico Hulkenberg']
Current Team Value: 103.5
Current Available Value: 0.6
Total Number of Team Combinations: 697680
Total Number of Team Combinations I can afford: 439120
Explored all of the valid 439120 teams.

=== TEAM AT POSITION 1 WITH SCORE 177 ===
Constructor: ('Red Bull Racing-RBPT', 'Ferrari')
Drivers: ('Sergio Perez', 'Nico Hulkenberg', 'Daniel Ricciardo', 'Pierre Gasly', 'Gu